In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2 as cv
from sklearn.model_selection import train_test_split

file_path = "C:\\workspace\\airfoil"

def img_read(file_path, file):
    img = cv.imread(file_path + '//' + file, cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, (56, 56))
    # 이미지 값을 0 또는 255로 변환
    _, img = cv.threshold(img, 127, 255, cv.THRESH_BINARY)
    return img

def get_data():
    files = os.listdir(file_path)
    X = []
    for file in files:
        img = img_read(file_path, file)
        X.append(img / 255.0)  # 값을 0.0 또는 1.0으로 정규화
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=1, shuffle=True)
    X_train = np.expand_dims(X_train, axis=3)
    X_test = np.expand_dims(X_test, axis=3)
    return np.array(X_train), np.array(X_test)

X_train, X_test = get_data()
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

def build_generator():
    model = Sequential()

    model.add(Dense(128 * 14 * 14, activation="relu", input_dim=100))
    model.add(Reshape((14, 14, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, kernel_size=3, padding="same", activation='sigmoid'))  # sigmoid 활성화 함수 사용

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

generator = build_generator()
generator.summary()

def build_discriminator():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(56, 56, 1), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=(56, 56, 1))
    validity = model(img)

    return Model(img, validity)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.summary()

discriminator.trainable = False

z = Input(shape=(100,))
img = generator(z)
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
combined.summary()

def train(epochs, batch_size=128, sample_interval=50):
    REAL = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    history = []

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, REAL)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = combined.train_on_batch(noise, REAL)

        history.append({"D": d_loss[0], "G": g_loss})

        if epoch % sample_interval == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
            sample_images(epoch)
    
    return history

def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = (gen_imgs > 0.5).astype(int)  # 0 또는 1로 변환
    gen_imgs = gen_imgs * 255  # 0 또는 255로 변환

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

history = train(epochs=10001, batch_size=32, sample_interval=1000)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:\\workspace\\airfoil'